In [ ]:
# adjust dyad position +/- 500
import General
from pathlib import Path

General.adjust_dyad_positions(
    dyad_file=Path('/media/cam/Data9/CortezAnalysis/Cam_calls/nucleosome_stuff/dyads.bed')
)

In [ ]:
# getfasta ran on the dyads to get the context
import BedtoolsCommands
from pathlib import Path
terminal_out, file_location = BedtoolsCommands.bedtools_getfasta(
    bed_file=Path('/media/cam/Data9/CortezAnalysis/Cam_calls/nucleosome_stuff/dyads_plus-minus_1000.bed'),
    fasta_file=Path('/media/cam/Data9/CortezAnalysis/pipeline_files/hg19.fa')
)
print(terminal_out)

In [ ]:
# filter lines with 'N'
import General
from pathlib import Path

General.filter_lines_with_n(
    dyad_fasta = Path('/media/cam/Data9/CortezAnalysis/Cam_calls/nucleosome_stuff/dyads_plus-minus_1000_hg19_fasta.fa'),
    dyad_bed=Path('/media/cam/Data9/CortezAnalysis/Cam_calls/nucleosome_stuff/dyads.bed')
)


In [ ]:
import General
from pathlib import Path

General.filter_acceptable_chromosomes(
    Path('/media/cam/Data9/CortezAnalysis/Cam_calls/nucleosome_stuff/dyads_plus-minus_1000_filtered.bed'),
    genome = 'human'
    )

In [ ]:
# use linux to sort a file based on the first 3 and sixth column
import General
from pathlib import Path
General.check_and_sort(Path('/media/cam/Data9/CortezAnalysis/Cam_calls/nucleosome_stuff/dyads_filtered.bed'))
# General.check_and_sort(Path('/media/cam/Data9/CortezAnalysis/Cam_calls/Analysis/vcf_files/concat/KM_treated_filtered.bed'))

In [ ]:
# counts all the different positions in the dyad file
import multiprocessing as mp
import Statistics
from pathlib import Path

if __name__ == '__main__':
    mp.freeze_support()
    fasta_counter = Statistics.DyadFastaCounter(
        Path('/media/cam/Data9/CortezAnalysis/Cam_calls/nucleosome_stuff/dyads_plus-minus_1000_hg19_fasta_filtered.fa')
        )

In [ ]:
# counts all the different positions in the dyad file
import multiprocessing as mp
import Statistics
from pathlib import Path

if __name__ == '__main__':
    mp.freeze_support()
    fasta_counter = Statistics.MutationIntersector(
        mutation_file = Path('/media/cam/Data9/CortezAnalysis/Cam_calls/8-oxo-G_Mapping_Data/split-reads/joined_bed/SRR_67-68_sorted_sorted.bed'),
        dyad_file = Path('/media/cam/Data9/CortezAnalysis/Cam_calls/nucleosome_stuff/dyads_files/dyads_plus-minus_1000_filtered_sorted.bed')
    )

In [ ]:
import General
from pathlib import Path
General.vcf_snp_to_intermediate_bed(Path('/media/cam/Data9/CortezAnalysis/Cam_calls/Analysis/vcf_files/concat/KM_treated.vcf'))
General.expand_context_custom_bed(Path('/media/cam/Data9/CortezAnalysis/Cam_calls/Analysis/vcf_files/concat/KM_treated.tmp'),
Path('/media/cam/Data9/CortezAnalysis/pipeline_files/hg19.fa'))

In [ ]:
from threading import *
import multiprocessing as mp
import Statistics
from pathlib import Path
class StartUp(Thread):
    def __init__(self):
        super(StartUp, self).__init__()
    def run(self):
        fasta_counter = Statistics.MutationIntersector(
            mutation_file = Path('/media/cam/Data9/CortezAnalysis/Cam_calls/Analysis/vcf_files/concat/KM_treated_filtered_sorted.bed'),
            dyad_file = Path('/media/cam/Data9/CortezAnalysis/Cam_calls/nucleosome_stuff/dyads_files/dyads_plus-minus_1000_filtered_sorted.bed')
        )

test = StartUp()
test.start()

In [ ]:
with open('/media/cam/Data9/CortezAnalysis/Cam_calls/8-oxo-G_Mapping_Data/split-reads/joined_bed/SRR_69-70_sorted.bed') as f:
    with open('/media/cam/Data9/CortezAnalysis/Cam_calls/8-oxo-G_Mapping_Data/split-reads/joined_bed/SRR_69-70_adjusted.bed', 'w') as o:
        for line in f:
            tsv = line.strip().split('\t')
            new_line = f'{tsv[0]}\t{str(int(tsv[1])+1)}\t{str(int(tsv[2])-1)}\t{tsv[3]}\t{tsv[4]}\t{tsv[5]}\t{tsv[6]}\n'
            o.write(new_line)

In [ ]:
with open('/home/cam/Documents/UV_Data/MELA-AU_trinuc_context_mutations.bed') as f:
    with open('/home/cam/Documents/UV_Data/MELA-AU_trinuc_context_mutations.bed6', 'w') as o:
        for line in f:
            tsv = line.strip().split('\t')
            new_line = f'{tsv[0]}\t{tsv[1]}\t{tsv[2]}\t.\t0\t{tsv[5]}\t{tsv[3]}\t{tsv[4]}\n'
            o.write(new_line)

In [ ]:
import Tools
import pandas as pd
with open('/media/cam/Data9/CortezAnalysis/Cam_calls/nucleosome_stuff/64-66_intersect.txt') as intersect:
    context_list = Tools.contexts_in_iupac('NNN')
    counts = {i: {key: 0 for key in context_list} for i in range(-1000,1001)}
    for line in intersect:
        tsv = line.strip().split('\t')
        bed_start = int(tsv[1])
        bed_end = int(tsv[2])
        dyad = int((int(tsv[8])+int(tsv[9]))/2)
        context = tsv[6]
        position = bed_start-dyad
        if position in [-1001, 1001]: continue
        counts[position][context] += 1
    df = pd.DataFrame.from_dict(counts, orient='index')
    df.columns = context_list
    df.index.name = 'Position'
    df.to_csv('64-66_counts.txt', sep='\t')